In [1]:
import torch 
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
#create a rnn
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

In [23]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [24]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [25]:
model1 = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=learning_rate)

In [27]:
model1.train()
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        

        # forward
        scores = model1(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

RuntimeError: Expected target size [64, 10], got [64]

In [15]:
model1.eval()
def check_accuracy(loader, model):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for data, targets in test_loader:
            data = data.to(device=device).squeeze(1)
            targets = targets.to(device=device)
            scores = model1(data)

            _, predictions = scores.max(1)
            n_samples += targets.size(0)
            n_correct += (predictions == targets).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy: {acc}')

In [ ]:
print("Checking accuracy on training and test data")
check_accuracy(train_loader, model1)
check_accuracy(test_loader, model1)

Checking accuracy on training and test data
Accuracy: 98.42
Accuracy: 98.42
